In [1]:
import numpy as np
from sklearn.cluster import KMeans
from gensim.models import Word2Vec
from tabulate import tabulate
from collections import Counter

In [2]:
dataset = ["I love playing football on the weekends",
 "I enjoy hiking and camping in the mountains",
 "I like to read books and watch movies",
 "I prefer playing video games over sports",
 "I love listening to music and going to concerts"]

In [3]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

# Function to preprocess a sentence
def preprocess(text):
    text = text.lower()  # 1. Lowercase
    text = re.sub(r'[^\w\s]', '', text)  # 2. Remove punctuation
    words = text.split()
    words = [word for word in words if word not in stop_words]  # 3. Remove stopwords
    return ' '.join(words)

# Apply to dataset
preprocessed_dataset = [preprocess(sentence) for sentence in dataset]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jeyas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
tokenized_dataset = [doc.split() for doc in preprocessed_dataset]
word2vec_model = Word2Vec(sentences=tokenized_dataset, vector_size=100, 
window=5, min_count=1, workers=4)

In [8]:
X = np.array([np.mean([word2vec_model.wv[word] for word in doc.split() if word in 
word2vec_model.wv], axis=0) for doc in preprocessed_dataset])

In [11]:
k = 2 # Define the number of clusters
km = KMeans(n_clusters=k)
km.fit(X)
# Predict the clusters for each document
y_pred = km.predict(X)
# Tabulate the document and predicted cluster
table_data = [["Document", "Predicted Cluster"]]
table_data.extend([[doc, cluster] for doc, cluster in zip(preprocessed_dataset, y_pred)])
print(tabulate(table_data, headers="firstrow"))

Document                               Predicted Cluster
-----------------------------------  -------------------
love playing football weekends                         0
enjoy hiking camping mountains                         1
like read books watch movies                           0
prefer playing video games sports                      0
love listening music going concerts                    1


C:\Users\jeyas\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [ ]:
# Calculate purity
total_samples = len(y_pred)
cluster_label_counts = [Counter(y_pred)]
purity = sum(max(cluster.values()) for cluster in cluster_label_counts) / total_samples
print("Purity:", purity)